In [8]:
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
import re

import matplotlib.pyplot as plt

## Unless otherwise stated, the following datasets are sourced from this website:
https://www.gate.ezv.admin.ch/swissimpex/index.xhtml

## Main question driving the project: How can a Swiss consumer, based on all the factors that go into the carbon emissions of food production, best adapt their diet to reduce their carbon foodprint?

As is slowly becoming a widely-known fact, the meat and animal product industry is significantly more carbon-intensive than plant-based foods. The amount of CO2 produced per kg (or even kcal) of meat/animal product is much higher than the amount of CO2 for the same mass/energy equivalent of plants (e.g. fruits, vegetables, legumes). One might think, then, that the best way to reduce one's "carbon foodprint" (carbon footprint due to food production) is to take on a vegetarian or vegan diet.

What is intriguing is that this may not be the answer in all cases. Based on where food is sourced, what time of year it is grown, and how it is produced, the carbon emissions can change drastically. A hot house tomato grown out of season can produce less carbon than pasture-fed goat meat, for example. 

An important question that data can answer, then, is how an eco-friendly citizen can best adapt their diet, based on their specific living circumstances, to reduce their carbon foodprint. Here, we will perform an in-depth analysis on Switzerland, providing a consumer with the knowledge necessary to produce less carbon emissions due to their diet. This will include an analysis of carbon intensities by food type and the various types and sources of food that are imported into Switzerland. After all, since Switzerland has a fairly dense population for its small size, it imports a significant amount of its food:

"According to the Federal Agriculture Office, Switzerland’s gross self-sufficiency rate in 2015 was 59%. The degree of self-sufficiency is defined as the ratio of domestic production to total domestic consumption. With imported animal feed taken into account, the net level of self-sufficiency was 51% that year.

But a closer look at the data reveals major differences across products. The country has been able to produce almost 100% of its animal foodstuffs for years, yet has managed only about 40% self-sufficiency for plant-based foods, a rate that fluctuates year on year as harvests are highly weather-dependent." (From swissinfo.ch)

Depending on where and what types of food Switzerland is importing its non-meat food products from, this raises the possibility that a Swiss citizen might have a smaller carbon foodprint by eating more meat, contradicting the typical recommendation. Would living with a vegan diet require an excessive amount of imported foods, outweighting the carbon offset of not eating meat? Or is there a large enough amount of imported livestock feed that eating a plant-based diet is still more sustainable? 

Let's start by looking at Swiss food imports on a macro scale.

In [9]:
# this sheet is food, beverages, and tobacco
imported_goods = pd.read_excel('../data/nature-of-goods-imports.xlsx', skiprows=5, sheet_name='01.1')
imported_goods.dropna(how='all',inplace=True)
imported_goods.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], inplace=True) #empty columns
imported_goods.rename({'Unnamed: 0':'Year', 'Unnamed: 1':'Commercial Partner'}, inplace=True, axis=1)
imported_goods['Commercial Partner'] = imported_goods['Commercial Partner'].str.strip() # get rid of leading whitespace
imported_goods.head()

,Year,Commercial Partner,Quantity (kg),Value (CHF),Value +/- %
0,2017,Total trade,4.125947e+09,1.058994e+10,4.5
1,2017,Afghanistan,3.300000e+01,1.229620e+05,244.1
2,2017,Albania,3.444010e+05,3.499380e+05,-26.7
3,2017,Algeria,9.049800e+04,4.043350e+05,0.1
4,2017,Amer. Virgin,5.000000e+01,2.793000e+03,-90.6


Let's look at how much of the food, beverages, and tobacco that Switzerland imports comes from its nearest neighboring countries. The farther a country is, the more carbon emissions it would cost to import that food.

In [10]:
# I just hand-wrote this list, it doesn't actually mean anything. Just picked nearby countries
nearby_countries = ['Portugal', 'Spain', 'France', 'Germany', 'Italy', 'Austria', 'Belgium', 'Netherlands', 'Czech Republic', 'Slovenia', 'Croatia']
imported_goods_nearby = imported_goods[imported_goods['Commercial Partner'].isin(nearby_countries)].copy()
imported_goods_nearby.head()

,Year,Commercial Partner,Quantity (kg),Value (CHF),Value +/- %
10,2017,Austria,205839135.0,3.633062e+08,11.3
17,2017,Belgium,53206197.0,1.654063e+08,2.1
44,2017,Croatia,1937165.0,7.261209e+06,18.4
48,2017,Czech Republic,62643717.0,4.549908e+07,16.5
63,2017,France,625539021.0,1.281604e+09,3.6


In [11]:
total_nearby_imported = imported_goods_nearby['Quantity (kg)'].sum()
100*total_nearby_imported/imported_goods.iloc[0][2]

74.21443433630381

Approximately 74% of Switzerland's imported food, beverage, and tobacco products come from fairly nearby countries. 

What about Switzerland's imported feedstuffs for animals? (If the Swiss import a lot of feed, that contributes to an even higher carbon intensity for raising meat)

In [12]:
# this sheet is feeding stuffs for animals
imported_feed = pd.read_excel('../data/nature-of-goods-imports.xlsx', skiprows=5, sheet_name='01.2')
imported_feed.dropna(how='all',inplace=True)
imported_feed.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], inplace=True) #empty columns
imported_feed.rename({'Unnamed: 0':'Year', 'Unnamed: 1':'Commercial Partner'}, inplace=True, axis=1)
imported_feed['Commercial Partner'] = imported_feed['Commercial Partner'].str.strip() # get rid of leading whitespace
imported_feed.head()

,Year,Commercial Partner,Quantity (kg),Value (CHF),Value +/- %
0,2017,Total trade,1.805791e+09,790556375.0,5
1,2017,Albania,7.400000e+01,857.0,26.6
2,2017,Argentina,1.153685e+06,396099.0,268.2
3,2017,Australia,5.040000e+02,1627.0,-88.5
4,2017,Austria,1.028771e+08,40044159.0,17.3


In [13]:
imported_feed_nearby = imported_feed[imported_feed['Commercial Partner'].isin(nearby_countries)].copy()
total_nearby_imported_feed = imported_feed_nearby['Quantity (kg)'].sum()
100*total_nearby_imported_feed/imported_feed.iloc[0][2]

73.97204548162978

Once again, nearly three-quarters is imported. This significantly raises the carbon emissions of meat grown and consumed in Switzerland.

Let's now look at how much meat Switzerland imports.

In [14]:
# should download this data for 2017 instead to match swiss emissions and consumption data
# "selected" meat imports since there is way more detail available, I just didn't download it
meat_imported = pd.read_excel('../data/selected-meat-imports.xlsx', skiprows=5, sheet_name='02')
meat_imported.dropna(how='all',inplace=True)
meat_imported.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], inplace=True)
meat_imported.rename({'Unnamed: 0':'Year', 'Unnamed: 1':'Commercial Partner'}, inplace=True, axis=1)
meat_imported.head()

,Year,Commercial Partner,Quantity (kg),Value (CHF),Value +/- %
0,2018,Total trade,77042136.0,688620381.0,-2
1,2018,Albania,141.0,5676.0,*
2,2018,Argentina,1460763.0,16806383.0,13.4
3,2018,Australia,2801755.0,60160817.0,-10.1
4,2018,Austria,6358938.0,44545918.0,-10.4


As we can see from the above table, Switzerland imported 77 million kg of meat and edible offal in 2018 (equivalent to 77 thousand tonnes). The breakdowns by country are also provided.

Now let's look at what Switzerland's population _consumes_. 

In [15]:
# Note: all of these values are from 2017 (earlier years are also available)
food_consumption_types = pd.read_excel(r'../data/food_consumption_by_type_of_food.xlsx', skiprows=8)
food_consumption_types.dropna(inplace=True)
food_consumption_types.rename({'1000 t':'Quantity, total, 1000 tonnes', 'kg ':'Quantity, kg/person/year', ' t':'Protein, total, tonnes', 'g':'Protein, g/person/day', '%':'Protein, % indigenous production', 'TJ':'Energy intake, total, TJ', 'KJ':'Energy intake, kj/person/day', '%.1':'Energy intake, % indigenous production'}, inplace=True, axis=1)
food_consumption_types.head()

,,"Quantity, total, 1000 tonnes","Quantity, kg/person/year","Protein, total, tonnes","Protein, g/person/day","Protein, % indigenous production","Energy intake, total, TJ","Energy intake, kj/person/day","Energy intake, % indigenous production"
2,"Denrées alimentaires, total",7157.034169,841.014590,302451.392778,97.371792,70.553057,39794.586230,12811.546844,59.043218
4,Denrées alimentaires\nvégétales,4503.066844,529.150039,121985.710268,39.272318,41.448430,28153.732943,9063.867792,42.657761
5,Produits à base de céréales,767.003856,90.129713,71022.714316,22.865191,55.579067,8145.924472,2622.514841,58.419713
6,"Pommes de terre, etc.",364.760695,42.862596,5262.916468,1.694354,91.157487,853.361540,274.732882,88.965659
7,Amidons,29.015108,3.409530,128.987886,0.041527,0.000000,414.978520,133.598995,0.000000


Specifically, let's look at meat from the above table. How much meat did Swiss people consume in 2017?

In [16]:
food_consumption_types.iloc[17]

                                           Viande
Quantity, total, 1000 tonnes              413.333
Quantity, kg/person/year                  48.5702
Protein, total, tonnes                    82059.2
Protein, g/person/day                     26.4183
Protein, % indigenous production          76.1266
Energy intake, total, TJ                  3724.74
Energy intake, kj/person/day              1199.15
Energy intake, % indigenous production    85.7077
Name: 21, dtype: object

As can be seen, the Swiss consumed 413 thousand tonnes, but they imported 77 thousand tonnes. This means most of the Swiss meat is produced within the country. (This calculates to be 81% of consumed Swiss meat to be domestically produced/non-imported)

Some other important facts, as provided by an article on swissinfo.ch: Switzerland produces more than enough milk and milk products to satisfy its needs, but has to import the majority of its fruits, vegetables, and nuts. Animal feed is one of the major imports, as well as staples like rice and wheat.

##### How does this affect emissions?
The next thing we can look at is how intensive (in terms of carbon emissions) it is for Switzerland to produce certain types of food.

In [17]:
# this data is from FAO, emissions intensities
swiss_emis = pd.read_csv(r'../data/swiss_emissions_intensities.csv')
swiss_emis = swiss_emis[swiss_emis['Unit'] == 'kg CO2eq/kg product']
swiss_emis

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,1718,Cereals excluding rice,2016,2016,kg CO2eq/kg product,0.3031,Fc,Calculated data
3,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,867,"Meat, cattle",2016,2016,kg CO2eq/kg product,12.4773,Fc,Calculated data
6,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,882,"Milk, whole fresh cow",2016,2016,kg CO2eq/kg product,0.5491,Fc,Calculated data
9,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,1017,"Meat, goat",2016,2016,kg CO2eq/kg product,19.9347,Fc,Calculated data
12,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,1020,"Milk, whole fresh goat",2016,2016,kg CO2eq/kg product,0.3706,Fc,Calculated data
15,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,977,"Meat, sheep",2016,2016,kg CO2eq/kg product,18.1379,Fc,Calculated data
18,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,982,"Milk, whole fresh sheep",2016,2016,kg CO2eq/kg product,0.6072,Fc,Calculated data
21,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,1058,"Meat, chicken",2016,2016,kg CO2eq/kg product,0.3371,Fc,Calculated data
24,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,1062,"Eggs, hen, in shell",2016,2016,kg CO2eq/kg product,0.5305,Fc,Calculated data
27,EI,Emissions intensities,211,Switzerland,71761,Emissions intensity,1035,"Meat, pig",2016,2016,kg CO2eq/kg product,1.5056,Fc,Calculated data


# Where Brooke sees this going from here
Next, we'd have to look at what types of food Switzerland typically imports from other countries, then look at the emissions intensities for those foods in those countries. It would be helpful to have a dataset on a typical Swiss diet (e.g. 25% meat, 30% fruit, etc.), but I haven't looked for that yet and I don't know if it exists. If it doesn't, we can approximate it somehow through recommended nutritional guidelines or something. 

I don't know how we're going to approach calculating emissions for imported animals and meat (meaning, also including the carbon emissions of transport). I haven't thought about/researched that yet.

The data from Swiss-Impex (linked in the top of the notebook) is extremely detailed and has a lot more information to give. Feel free to check it out and get a sense of other data that we could use from there.